In [9]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


In [10]:
df_eng_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_Mar262021\ESA_website_ENG_old_projects.csv")
df_eng_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads\ESA_website_ENG_new_projects.csv")

df_fra_old_projects = pd.read_csv("G:\ESA_downloads\download_Bingjie_May062021_fra\ESA_website_FRA.csv")
df_fra_new_projects = pd.read_csv("G:\ESA_downloads\phase_2_download_files\ESA_downloads_fra\ESA_website_FRA_new_projects.csv")

df_eng_all_projects = pd.read_csv("G:\ESA_downloads\ESA_with_VCs\ESA_website_ENG.csv")

C:\Users\ThibJacq\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (52) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [11]:
df_eng_old_projects.head()

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,ID,Project Download Path,Table Download Path,Good Quality
0,Figure 13.1-1 EnCana Ekwan Pipeline,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",26,107.0,1.41,Yes,15151,NaN,NaN,NaN
1,Figure 13.3-1 CEA Framework,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",41,107.0,1.41,Yes,15152,NaN,NaN,NaN
2,Figure 13.4-1 Temperature Normals Measured at ...,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",44,107.0,1.41,Yes,15153,NaN,NaN,NaN
3,Figure 13.4-2 Mean Monthly Rainfall and Number...,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",45,107.0,1.41,Yes,15154,NaN,NaN,NaN
4,Figure 13.4-3 Mean Monthly Snowfall and Number...,Figure,Application to Construct and Operate Ekwan Pip...,Ekwan,2003-03-17,EnCana Ekwan Pipeline Inc.,Gas,A0H8C0 - 13.0 EIA - Section 13.1 to 13.6,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,A0H8C0,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),"Alberta, British Columbia",GH-1-2003,AXYS Environmental Consulting Ltd.,Operating,GC-108,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/2...,"Section 13.1: Introduction, Section 13.1: Proj...",1.0,"Land, Air, Vegetation",46,107.0,1.41,Yes,15155,NaN,NaN,NaN


In [21]:
def merge_dfs_with_matching_specific_columns(left_df, right_df, left_on, right_on):
    """Merge two dataframes based on common columns and only keep rows where
    the values match.
    """
    merged_df = pd.merge(left_df, right_df, how='inner', left_on=left_on, right_on=right_on, suffixes=('_x', ''))
    merged_df.drop(merged_df.filter(regex='_x$').columns, axis=1, inplace=True)
    return merged_df

left_cols = ['Title', 'Document Number', 'PDF Page Number']
right_cols = ['Title', 'Document Number', 'PDF Page Number']

df_eng_old_merged = merge_dfs_with_matching_specific_columns(df_eng_old_projects, df_eng_all_projects, left_on=left_cols, right_on=right_cols)
df_eng_new_merged = merge_dfs_with_matching_specific_columns(df_eng_new_projects, df_eng_all_projects, left_on=left_cols, right_on=right_cols)

In [25]:
df_eng_new_merged.tail()

,Title,Document Number,PDF Page Number,filename,Hearing Order,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Data ID,PDF Download URL,Application Type (NEB Act),Pipeline Location,Hearing order,Consultant Name,Pipeline Status,Regulatory Instrument(s),Application URL,Decision URL,ESA Section(s),ESA Section(s) Index,ESA Section(s) Topics,PDF Page Count,PDF Size,PDF Outline,"Landscape, terrain, and weather",Soil,Plants,Water,Fish,Wetlands,Wildlife,Species at Risk,Greenhouse gas emissions,Air emissions,Noise,Electricity and electromagnetism,Proximity to people,"Archaeological, paleontological, historical, and culturally significant sites and resources",Human access to boats and waterways,"Indigenous land, water, and air use",Impact to social and cultural well-being,Impact to human health and viewscapes,"Social, cultural, economic infrastructure and services",Economic Offsets and Impact,Environmental Obligations,Treaty and Indigenous Rights,ID,Project Download Path,Table Download Path,Good Quality
639,"Figure 13.18 Occupational Employment 2016, by ...",A7J7C9,432,NaN,GH-002-2020,Figure,Application for the NGTL West Path Delivery 20...,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,4003703,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,NaN,7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILD...,NaN,NaN,612.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16296,NaN,NaN,NaN
640,"Figure 13.19 Occupational Employment 2016, by ...",A7J7C9,434,NaN,GH-002-2020,Figure,Application for the NGTL West Path Delivery 20...,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,4003703,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,NaN,7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILD...,NaN,NaN,612.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16297,NaN,NaN,NaN
641,"Figure 13.20 Occupational Employment 2016, by ...",A7J7C9,435,NaN,GH-002-2020,Figure,Application for the NGTL West Path Delivery 20...,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,4003703,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,NaN,7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILD...,NaN,NaN,612.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16298,NaN,NaN,NaN
642,Figure 14.1 Locations of Nearest Populated Lan...,A7J7C9,499,NaN,GH-002-2020,Figure,Application for the NGTL West Path Delivery 20...,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,Gas,C09063-11 NGTL WPD 2023_ESA_Part 2_Sections 7 ...,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,4003703,https://apps.cer-rec.gc.ca/REGDOCS/File/Downlo...,Large Projects (over 40 km),Alberta,GH-002-2020,Stantec Consulting Ltd,Applied,NaN,https://apps.cer-rec.gc.ca/REGDOCS/Item/View/3...,NaN,7.0 ASSESSMENT OF EFFECTS ON WILDLIFE AND WILD...,NaN,NaN,612.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16299,NaN,NaN,NaN
643,Figure 14.2 Locations of Nearest Populated Lan...,A7J7C9,500,NaN,GH-002-2020,Figure,Application for the NGTL West Path Delivery 20...,West Path Delivery 2023,2020-10-22,NOVA Gas Transmission Ltd.,G

In [ ]:
df_fra_old_merged = pd.merge(df_eng_old_merged, df_fra_old_projects, how='inner', on=['ID'])
df_fra_new_merged = pd.merge(df_eng_new_merged, df_fra_new_projects, how='inner', on=['ID'])